### :
    把内存中训练好的模型参数存储在硬盘上供后续读取使用

### 读写Tensor:
    可以直接使用 save 函数和 load 函数分别存储和读取 Tensor

In [1]:
import torch
import numpy as np

In [9]:
x = torch.ones(3)

In [10]:
torch.save(x,'./save/x.pt')

In [11]:
x2 = torch.load('./save/x.pt')
x2

tensor([1., 1., 1.])

In [12]:
# 还可以存储一个 Tensor 列表并读回内存
y = torch.zeros(4)
torch.save([x,y],'./save/x_y_list.pt')
xy_list = torch.load('./save/x_y_list.pt')
xy_list

[tensor([1., 1., 1.]), tensor([0., 0., 0., 0.])]

In [14]:
# 读取一个从字符串映射到 Tensor 的字典
torch.save({'x':x,'y':y},'./save/xy_dict.pt')
xy_dict = torch.load('./save/xy_dict.pt')
xy_dict

{'x': tensor([1., 1., 1.]), 'y': tensor([0., 0., 0., 0.])}

### 读写模型

##### state_dict:
    state_dict是一个从参数名称隐射到参数Tesnor的字典对象

In [19]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.hidden = torch.nn.Linear(3,2)
        self.act = torch.nn.ReLU()
        self.output = torch.nn.Linear(2,1)
    def forward(self,x):
        out = self.output(self.act(self.hidden(x)))
        return out

In [20]:
net = MLP()
net

MLP(
  (hidden): Linear(in_features=3, out_features=2, bias=True)
  (act): ReLU()
  (output): Linear(in_features=2, out_features=1, bias=True)
)

In [21]:
net.state_dict() 
# 只有具有可学习参数的层(卷积层、线性层等)才有state_dict中的条目
# 优化器(optim)也有一个state_dict，其中包含关于优化器状态以及所使用的超参数的信息

OrderedDict([('hidden.weight',
              tensor([[ 0.0756,  0.4586, -0.5424],
                      [ 0.3426, -0.4228,  0.5319]])),
             ('hidden.bias', tensor([ 0.0533, -0.4768])),
             ('output.weight', tensor([[-0.6741, -0.4605]])),
             ('output.bias', tensor([0.0383]))])

#### 保存和加载模型:
    两种常见：
        仅保存和加载模型参数(state_dict)： => 推荐优先使用
            保存：torch.save(model.state_dict(), PATH) # 推荐的文件后缀名是pt或pth
            加载：
                model = TheModelClass(*args, **kwargs)
                model.load_state_dict(torch.load(PATH))
        保存和加载整个模型：
            保存：torch.save(model, PATH)
            加载：model = torch.load(PATH)

In [22]:
# 保存和加载state_dict(推荐方式)
x = torch.randn(2,3)
out = net(x)

In [23]:
PATH = './model/net.pt'
torch.save(net.state_dict(),PATH)

In [24]:
net_2 = MLP()
net_2.load_state_dict(torch.load(PATH))

out_2 = net_2(x)
out_2 == out # 这 net 和 net2 都有同样的模型参数，那么对同一个输入 x 的计算结果将会是一样的

tensor([[True],
        [True]])